In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import mglearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import random
from sklearn.utils import shuffle

from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils                         # NumPy related tools
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
import numpy as np

import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
import tensorflow as tf


In [2]:
data = pd.read_csv("Energi_Viborg_Dandas_data.csv")

In [3]:
#drop columns not needed after asking the company about the meaning of these features

columns_to_be_removed = ['ID', 'mslink', 'XKoordinat','YKoordinat','LedningID','Dobbeltled','EjerKompon','SystemKode','KategoriAf','DatoUdf']
data=data.drop(columns_to_be_removed,axis='columns')

In [4]:
# in the column DatoSaneri is the date of repairing and if there is no date it means it is not repaired

data['DatoSaneri'].fillna(0, inplace=True)

In [5]:
# take only the pipes that are broken(by TV insection) now and the repaired ones

data_with_TVObsAndSaneri = data[data['TVObsKode'].isin([1]) | data['DatoSaneri'] > 0]

In [6]:
#get around 2077 rows with not broken pipes

data_not_broken = data[~data['TVObsKode'].isin([0]) | data['DatoSaneri'] == 0]
data_not_broken = data_not_broken.sample(n=2077) 

In [7]:
frames = [data_with_TVObsAndSaneri, data_not_broken]
  
data_final = pd.concat(frames)
data_final

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri
36,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0
42,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
43,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0
64,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
65,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8131,14.70,13.670000,62.66,16.437919,250.0,1.0,1964.0,1,0,0.0,2010,2014,0.0
7321,38.27,37.920716,53.48,6.531113,180.0,4.0,1995.0,1,0,0.0,2010,2014,0.0
8745,32.28,31.790000,31.77,15.423355,600.0,4.0,2007.0,1,0,0.0,2010,2014,0.0
14629,31.02,30.090000,40.53,22.945966,400.0,1.0,1965.0,1,0,0.0,2010,2014,0.0


In [8]:
data = data_final

In [9]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [10]:
#get data copied
datacopy = data


# add  age column

#get current year
from datetime import date
now = date.today().year


def age_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] > 0) :
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])

datacopy['Age'] = datacopy.apply(age_df, axis = 1)

In [11]:
# add a column 'PipeStatus'
# 1 as broken and 0 as not broken

def broken_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] < (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 1
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] >= (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 0
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return 1
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return 0
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return 0

datacopy['PipeStatus'] = datacopy.apply(broken_df, axis = 1)

In [12]:
# datacopy = datacopy.sample(n=22) 
# datacopy

In [13]:
# data_fs= np.where(np.isnan(datacopy))
# data_fs
# row = datacopy.iloc[369] #index=1 => second row
# print(row)

In [14]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(datacopy.shape[0]))
datacopy = datacopy.dropna()
print("Number of rows after removing NaNs: {}".format(datacopy.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [15]:
#drop columns not needed after adding new features

columns_to_be_removed = ['DatoOprett', 'DatoOpdate']
datacopy=datacopy.drop(columns_to_be_removed,axis='columns')
datacopy[0:-1]

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Age,PipeStatus
36,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,1997.0,24.0,0
42,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
43,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,0.0,82.0,1
64,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
65,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,0.0,76.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,25.54,25.100000,54.82,8.026268,250.0,1.0,1943.0,1,0,0.0,0.0,78.0,0
8131,14.70,13.670000,62.66,16.437919,250.0,1.0,1964.0,1,0,0.0,0.0,57.0,0
7321,38.27,37.920716,53.48,6.531113,180.0,4.0,1995.0,1,0,0.0,0.0,26.0,0
8745,32.28,31.790000,31.77,15.423355,600.0,4.0,2007.0,1,0,0.0,0.0,14.0,0


In [16]:
# creating features set and target

columns_to_be_removed = ['PipeStatus']
data_features= datacopy.drop(columns_to_be_removed,axis='columns')
columns_to_be_removed = ['fra_kote','til_kote', 'Laengde','Fald','DiameterIn','MaterialeK','anlag_aar','TransportK','Funktionsk','TVObsKode','DatoSaneri','Age']
data_target=datacopy.drop(columns_to_be_removed,axis='columns')

In [17]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 4154
Number of rows after removing NaNs: 4154


In [18]:
data

,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri,Age,PipeStatus
36,34.72,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0,24.0,0
42,39.46,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0,82.0,1
43,39.71,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0,82.0,1
64,40.55,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0,76.0,1
65,40.38,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0,76.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8131,14.70,13.670000,62.66,16.437919,250.0,1.0,1964.0,1,0,0.0,2010,2014,0.0,57.0,0
7321,38.27,37.920716,53.48,6.531113,180.0,4.0,1995.0,1,0,0.0,2010,2014,0.0,26.0,0
8745,32.28,31.790000,31.77,15.423355,600.0,4.0,2007.0,1,0,0.0,2010,2014,0.0,14.0,0
14629,31.02,30.090000,40.53,22.945966,400.0,1.0,1965.0,1,0,0.0,2010,2014,0.0,56.0,0


In [19]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(data_features, data_target, random_state=42)

In [20]:
#scale the feature for neural networks
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
X_train_scaled = tf.expand_dims(X_train_scaled, axis=-1)
X_train_scaled = tf.expand_dims(X_train_scaled, axis=-1)

In [22]:
X_test_scaled = tf.expand_dims(X_test_scaled, axis=-1)
X_test_scaled = tf.expand_dims(X_test_scaled, axis=-1)

# CNN using activation function 'tanh'

In [23]:
#CNN
from keras.models import Sequential

model_1 = keras.models.Sequential()
model_1.add(keras.layers.Conv2D(32, (3, 3), padding='same',activation='tanh', input_shape=X_train_scaled[0].shape))
model_1.add(keras.layers.MaxPooling2D((1, 1)))
model_1.add(keras.layers.Flatten())
model_1.add(keras.layers.Dense(2, activation='softmax'))
# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model_1.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [24]:
%%time
model_1.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
98/98 [==============================] - 0s 1ms/step - loss: 0.3396 - accuracy: 0.8568
Epoch 2/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0623 - accuracy: 0.9884
Epoch 3/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0367 - accuracy: 0.9881
Epoch 4/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0293 - accuracy: 0.9891
Epoch 5/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0270 - accuracy: 0.9881: 0s - loss: 0.0329 - accuracy: 
Wall time: 1.18 s


# CNN using activation function 'relu'

In [25]:
#CNN
from keras.models import Sequential

model_2 = keras.models.Sequential()
model_2.add(keras.layers.Conv2D(32, (3, 3), padding='same',activation='relu', input_shape=X_train_scaled[0].shape))
model_2.add(keras.layers.MaxPooling2D((1, 1)))
model_2.add(keras.layers.Flatten())
model_2.add(keras.layers.Dense(20, activation='relu'))

model_2.add(keras.layers.Dense(2, activation='softmax'))
# compile model
opt = SGD(learning_rate=0.01, momentum=0.9)
model_2.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [26]:
%%time
model_2.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
98/98 [==============================] - 0s 2ms/step - loss: 0.2998 - accuracy: 0.8899
Epoch 2/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0303 - accuracy: 0.9881
Epoch 3/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0230 - accuracy: 0.9888
Epoch 4/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0221 - accuracy: 0.9900
Epoch 5/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0221 - accuracy: 0.9891
Wall time: 1.13 s


# CNN using activation function 'sigmoid' 

In [27]:
#CNN
from keras.models import Sequential

model_3 = keras.models.Sequential()
model_3.add(keras.layers.Conv2D(32, (3, 3), padding='same',activation='sigmoid', input_shape=X_train_scaled[0].shape))
model_3.add(keras.layers.MaxPooling2D((1, 1)))
model_3.add(keras.layers.Flatten())
model_3.add(keras.layers.Dense(100, activation='sigmoid', kernel_initializer='he_uniform'))
model_3.add(keras.layers.BatchNormalization())

model_3.add(keras.layers.Dense(2, activation='softmax'))
# compile model
opt = SGD(learning_rate=0.1, momentum=0.9)
model_3.compile(optimizer=opt, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [28]:
%%time
model_3.fit(X_train_scaled,y_train,epochs=5)

Epoch 1/5
98/98 [==============================] - 1s 2ms/step - loss: 0.0673 - accuracy: 0.9782
Epoch 2/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0266 - accuracy: 0.9894
Epoch 3/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0263 - accuracy: 0.9884
Epoch 4/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0324 - accuracy: 0.9891
Epoch 5/5
98/98 [==============================] - 0s 2ms/step - loss: 0.0280 - accuracy: 0.9881
Wall time: 1.35 s


# Fully connected

In [29]:
%%time
model_4 = keras.models.Sequential()
model_4.add(keras.layers.Dense(512, input_shape=(12,))) #(784,) is not a typo -- that represents a 784 length vector!
model_4.add(keras.layers.Activation('relu'))
model_4.add(keras.layers.Dropout(0.2))
#2nd hidden layer
model_4.add(keras.layers.Dense(512))
model_4.add(keras.layers.Activation('relu'))
model_4.add(keras.layers.Dropout(0.2))
#3rd
model_4.add(keras.layers.Dense(10))
model_4.add(keras.layers.Activation('softmax'))
model_4.summary()

model_4.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 512)               6656      
                                                                 
 activation (Activation)     (None, 512)               0         
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 512)               262656    
                                                                 
 activation_1 (Activation)   (None, 512)               0         
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 10)               

In [30]:
#scale the feature for neural networks
scaler = MinMaxScaler()
scaler.fit(X_train)
x_train_scaled = scaler.transform(X_train)
x_test_scaled = scaler.transform(X_test)

In [31]:
%%time
model_4.fit(x_train_scaled, y_train,batch_size=128, epochs=5,verbose=1)

Epoch 1/5
25/25 [==============================] - 0s 6ms/step - loss: 0.6656 - accuracy: 0.8372
Epoch 2/5
25/25 [==============================] - 0s 6ms/step - loss: 0.0283 - accuracy: 0.9872
Epoch 3/5
25/25 [==============================] - 0s 5ms/step - loss: 0.0222 - accuracy: 0.9881
Epoch 4/5
25/25 [==============================] - 0s 5ms/step - loss: 0.0262 - accuracy: 0.9859
Epoch 5/5
25/25 [==============================] - 0s 5ms/step - loss: 0.0224 - accuracy: 0.9881
Wall time: 1.02 s


# test accuracy for each model

In [32]:
%%time

# test the model on testing data
loss_1, acc_1 = model_1.evaluate(X_test_scaled, y_test, verbose = 1)
print('\nTest accuracy: ', acc_1)

33/33 [==============================] - 0s 1ms/step - loss: 0.0268 - accuracy: 0.9846

Test accuracy:  0.9846006035804749
Wall time: 197 ms


In [33]:
%%time

# test the model on testing data
loss_2, acc_2 = model_2.evaluate(X_test_scaled, y_test, verbose = 1)
print('\nTest accuracy: ', acc_2)

33/33 [==============================] - 0s 1ms/step - loss: 0.0274 - accuracy: 0.9846

Test accuracy:  0.9846006035804749
Wall time: 155 ms


In [34]:
%%time

# test the model on testing data
loss_3, acc_3 = model_3.evaluate(X_test_scaled, y_test, verbose = 1)
print('\nTest accuracy: ', acc_3)

33/33 [==============================] - 0s 1ms/step - loss: 0.3555 - accuracy: 0.8056

Test accuracy:  0.8055822849273682
Wall time: 209 ms


In [35]:
%%time

# test the model on testing data
loss_4, acc_4 = model_4.evaluate(x_test_scaled, y_test, verbose = 1)
print('\nTest accuracy: ', acc_4)

33/33 [==============================] - 0s 2ms/step - loss: 0.0237 - accuracy: 0.9846

Test accuracy:  0.9846006035804749
Wall time: 196 ms


In [37]:
%%time
#CNN model_1
y_pred_1 = model_1.predict(X_test_scaled).argmax(axis=1) #1d array
print("Accuracy on test set: {:.4f}".format(accuracy_score(y_test,y_pred_1)))
print("Confusion matrix: \n", confusion_matrix(y_test,y_pred_1))

Accuracy on test set: 0.9846
Confusion matrix: 
 [[819  16]
 [  0 204]]
Wall time: 125 ms


In [38]:

print(classification_report(y_test, y_pred_1))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       835
           1       0.93      1.00      0.96       204

    accuracy                           0.98      1039
   macro avg       0.96      0.99      0.98      1039
weighted avg       0.99      0.98      0.98      1039



In [40]:
%%time
#CNN model
y_pred_2 = model_2.predict(X_test_scaled).argmax(axis=1)
print("Accuracy on test set: {:.4f}".format(accuracy_score(y_test,y_pred_2)))
print("Confusion matrix: \n", confusion_matrix(y_test,y_pred_2))

Accuracy on test set: 0.9846
Confusion matrix: 
 [[819  16]
 [  0 204]]
Wall time: 131 ms


In [41]:
print(classification_report(y_test, y_pred_2))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       835
           1       0.93      1.00      0.96       204

    accuracy                           0.98      1039
   macro avg       0.96      0.99      0.98      1039
weighted avg       0.99      0.98      0.98      1039



In [42]:
%%time
y_pred_3 = model_3.predict(X_test_scaled).argmax(axis=1)
print("Accuracy on test set: {:.4f}".format(accuracy_score(y_test,y_pred_3)))
print("Confusion matrix: \n", confusion_matrix(y_test,y_pred_3))

Accuracy on test set: 0.8056
Confusion matrix: 
 [[835   0]
 [202   2]]
Wall time: 143 ms


In [43]:
print(classification_report(y_test, y_pred_3))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       835
           1       1.00      0.01      0.02       204

    accuracy                           0.81      1039
   macro avg       0.90      0.50      0.46      1039
weighted avg       0.84      0.81      0.72      1039



In [44]:
%%time

y_pred_4 = model_4.predict(x_test_scaled).argmax(axis=1)
print("Accuracy on test set: {:.4f}".format(accuracy_score(y_test,y_pred_4)))
print("Confusion matrix: \n", confusion_matrix(y_test,y_pred_4))

Accuracy on test set: 0.9846
Confusion matrix: 
 [[819  16]
 [  0 204]]
Wall time: 140 ms


In [45]:
print(classification_report(y_test, y_pred_4))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       835
           1       0.93      1.00      0.96       204

    accuracy                           0.98      1039
   macro avg       0.96      0.99      0.98      1039
weighted avg       0.99      0.98      0.98      1039

